In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification

# Specify the file path
file_path = "Merged_Aneurysm.csv"

# Load the CSV file into a Pandas DataFrame
data = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify the data has been loaded correctly
data.head()


,case_id,patient_id,aneurysmLocation,aneurysmType,ruptureStatus,age,sex,multipleAneurysms,sacVolume,sacSurfaceArea,...,maxPressure,meanPressure,maxSpeed,meanSpeed,minTAWSS,maxTAWSS,meanTAWSS,minOSI,maxOSI,meanOSI
0,C0001,P0085,ICA,LAT,U,53,F,False,125.259604,120.453515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C0002,P0112,ICA,LAT,U,35,F,False,119.002032,122.427263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C0003,P0115,ICA,TER,U,43,F,False,12.740609,23.480617,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C0004,P0116,ICA,TER,U,60,F,True,9.376307,18.084625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C0005,P0118,ICA,LAT,R,26,F,False,37.934454,54.844592,...,7665.0,3722.28,70.275319,13.19376,0.31251,176.44,20.90888,0.0,0.44672,0.001574


In [2]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

# Define the list of characteristics to be used
characteristics = [
    'age', 'sacVolume', 'sacSurfaceArea', 'vdcVolume', 'vdcSurfaceArea', 
    'sacSectionArea', 'ellipsoidVolume', 'ellipsoidMaxSemiaxis', 
    'ellipsoidMidSemiaxis', 'ellipsoidMinSemiaxis', 'sacCenterlineLength', 
    'ostiumSectionArea', 'ostiumSectionPerimeter', 'ostiumMinSize', 
    'ostiumMaxSize', 'ostiumShapeFactor', 'aspectRatio_star', 'sizeRatio_star', 
    'vesselDiameter', 'neckVesselAngle', 'sacVesselAngle', 'meanRadius', 
    'meanCurvature', 'meanTorsion', 'tortuosity', 'minRadius', 'maxRadius', 
    'maxCurvature', 'maxTorsion', 'bifurcationAngleInPlane', 
    'bifurcationAngleOutOfPlane', 'aneurysmLocation', 'aneurysmType'
]

# Subset the data to include only the specified characteristics and the target variable
data_subset = data[characteristics + ['ruptureStatus']]

# Separate the target variable
X = data_subset.drop(columns=['ruptureStatus'])
y = data_subset['ruptureStatus']

# Handle missing values for numerical and categorical features separately
numerical_columns = X.select_dtypes(include=np.number).columns
categorical_columns = X.select_dtypes(exclude=np.number).columns

# Impute missing values for numerical columns
imputer_numerical = SimpleImputer(strategy='mean')
X[numerical_columns] = imputer_numerical.fit_transform(X[numerical_columns])


# Encode categorical variables
if not categorical_columns.empty:
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    encoded_categorical = encoder.fit_transform(X[categorical_columns])
    encoded_categorical_columns = encoder.get_feature_names_out(categorical_columns)
    X = pd.concat([X.drop(columns=categorical_columns).reset_index(drop=True), 
                   pd.DataFrame(encoded_categorical, columns=encoded_categorical_columns)], axis=1)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Feature Scaling for X (numerical and encoded categorical)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Data preprocessing complete!")


Data preprocessing complete!


In [3]:
X.columns

Index(['age', 'sacVolume', 'sacSurfaceArea', 'vdcVolume', 'vdcSurfaceArea',
       'sacSectionArea', 'ellipsoidVolume', 'ellipsoidMaxSemiaxis',
       'ellipsoidMidSemiaxis', 'ellipsoidMinSemiaxis', 'sacCenterlineLength',
       'ostiumSectionArea', 'ostiumSectionPerimeter', 'ostiumMinSize',
       'ostiumMaxSize', 'ostiumShapeFactor', 'aspectRatio_star',
       'sizeRatio_star', 'vesselDiameter', 'neckVesselAngle', 'sacVesselAngle',
       'meanRadius', 'meanCurvature', 'meanTorsion', 'tortuosity', 'minRadius',
       'maxRadius', 'maxCurvature', 'maxTorsion', 'bifurcationAngleInPlane',
       'bifurcationAngleOutOfPlane', 'aneurysmLocation_BAS',
       'aneurysmLocation_ICA', 'aneurysmLocation_MCA', 'aneurysmType_TER'],
      dtype='object')

In [4]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200]
}
grid_search = GridSearchCV(estimator=xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                           param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_scaled, y_encoded)
xgb_model = grid_search.best_estimator_
xgb_model


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [5]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict

# Get the best model from GridSearchCV
best_model = grid_search.best_estimator_

# Make predictions using cross-validation for the best model
y_pred_prob_best = cross_val_predict(best_model, X_scaled, y_encoded, cv=3, method='predict_proba')[:, 1]

# Calculate AUC-ROC for the best model
auc_roc_best = roc_auc_score(y_encoded, y_pred_prob_best)

# Print the best model's parameters and its AUC-ROC
print(f"Best Model Parameters: {grid_search.best_params_}")
print(f"Mean Cross-Validation Accuracy: {grid_search.best_score_:.3f}")
print(f"AUC-ROC Score: {auc_roc_best:.3f}")


Best Model Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}
Mean Cross-Validation Accuracy: 0.738
AUC-ROC Score: 0.725


In [6]:
import shap

# Initialize SHAP Explainer
explainer = shap.Explainer(xgb_model, X_scaled)

# Compute SHAP values
shap_values = explainer(X_scaled)

print("SHAP values computation complete!")


[20:48:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


SHAP values computation complete!


In [ ]:
from umap.umap_ import UMAP
from hdbscan import HDBSCAN
import matplotlib.pyplot as plt

# Dimensionality reduction using UMAP with fixed randomness
shap_2d = UMAP(n_components=2, n_neighbors=15, min_dist=0, random_state=42).fit_transform(shap_values.values)

# Clustering using HDBSCAN
hdbscan = HDBSCAN(min_cluster_size=10, min_samples=5, metric='euclidean')
cluster_labels = hdbscan.fit_predict(shap_2d)

print("HDBSCAN Clustering complete!")

# Scatter plot of clusters
plt.figure(figsize=(10, 6))
plt.scatter(shap_2d[:, 0], shap_2d[:, 1], c=cluster_labels, cmap='viridis', s=10)
plt.colorbar(label="Cluster")
plt.title("Clusters of SHAP Values (2D UMAP)")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.show()


n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.


In [ ]:
from sklearn.metrics import silhouette_score

score = silhouette_score(shap_2d, cluster_labels)
print(f"Silhouette Score: {score}")


In [ ]:
import pandas as pd

cluster_data = pd.DataFrame(shap_values.values, columns=X.columns)
cluster_data['Cluster'] = cluster_labels
print(cluster_data.groupby('Cluster').mean())


In [46]:
import pandas as pd

# Create a DataFrame for SHAP values
shap_df = pd.DataFrame(shap_values.values, columns=X.columns)

# Add cluster labels to the SHAP DataFrame
shap_df['Cluster'] = cluster_labels


In [ ]:
# Group by clusters and calculate mean SHAP values
cluster_means = shap_df.groupby('Cluster').mean()

# Display the top SHAP features for each cluster
print(cluster_means.T)


In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Cluster and rupture status data
cluster_df = pd.DataFrame({
    'Cluster': cluster_labels,
    'RuptureStatus': y_encoded  # Assuming y_encoded corresponds to ruptureStatus
})

# Calculate rupture proportions by cluster
rupture_distribution = cluster_df.groupby('Cluster')['RuptureStatus'].mean()
cluster_sizes = cluster_df.groupby('Cluster')['RuptureStatus'].size()

# Create a contingency table for statistical testing
contingency_table = pd.crosstab(cluster_df['Cluster'], cluster_df['RuptureStatus'])

# Perform chi-squared test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

rupture_distribution, cluster_sizes, p_value


In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
from itertools import combinations

# Get unique cluster pairs
cluster_pairs = list(combinations(cluster_df['Cluster'].unique(), 2))

# Perform pairwise comparisons
results = []
for c1, c2 in cluster_pairs:
    sub_table = pd.crosstab(
        cluster_df[cluster_df['Cluster'].isin([c1, c2])]['Cluster'],
        cluster_df[cluster_df['Cluster'].isin([c1, c2])]['RuptureStatus']
    )
    if sub_table.values.min() < 5:  # Use Fisher's Exact Test for small counts
        _, p_value = fisher_exact(sub_table)
        test_type = "Fisher"
    else:  # Use Chi-Square for larger counts
        _, p_value, _, _ = chi2_contingency(sub_table)
        test_type = "Chi-Square"
    results.append({'Cluster Pair': (c1, c2), 'p-value': p_value, 'Test': test_type})

# Adjust for multiple comparisons (Bonferroni correction)
bonferroni_alpha = 0.05 / len(results)  # Adjusted significance level

# Output results
for result in results:
    result['Significant'] = result['p-value'] < bonferroni_alpha
    print(result)


In [ ]:
X.columns

In [ ]:
import shap

# Plot the summary plot
shap.summary_plot(shap_values.values, features=X_scaled, feature_names=X.columns)


In [ ]:
explainer = shap.Explainer(xgb_model, X_scaled, feature_names=X.columns)
shap_values = explainer(X_scaled)

shap.plots.heatmap(shap_values)


In [ ]:
# Mean SHAP values by cluster
cluster_summary = cluster_data.groupby('Cluster').mean()
print(cluster_summary)


In [ ]:
# Add cluster labels to the original dataset
original_data = data[characteristics].copy()
original_data['Cluster'] = cluster_labels
import seaborn as sns
original_data.columns

In [ ]:
# Dropping specified non-numeric features and 'age' from numeric columns
numeric_features_filtered = original_data.drop(columns=['Cluster', 'aneurysmLocation', 'aneurysmType', 'age'])

# Standardizing the numeric features
standardized_features_filtered = pd.DataFrame(
    scaler.fit_transform(numeric_features_filtered),
    columns=numeric_features_filtered.columns
)

# Adding the cluster labels back to the standardized data
standardized_features_filtered['Cluster'] = original_data['Cluster']

# Calculating the mean of each feature grouped by cluster
cluster_feature_means_filtered = standardized_features_filtered.groupby('Cluster').mean().T

# Creating the heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(cluster_feature_means_filtered, cmap='coolwarm', annot=False, cbar=True, linewidths=0.5)
plt.title("Heatmap of Standardized Features Stratified by Clusters")
plt.xlabel("Clusters")
plt.ylabel("Features")
plt.tight_layout()
plt.show()


In [ ]:
# Add cluster labels to the original dataset
original_data = data[characteristics].copy()
original_data['Cluster'] = cluster_labels

# Select only numeric columns
numeric_columns = original_data.select_dtypes(include=np.number)

# Calculate mean feature values for each cluster (true values)
true_cluster_summary = numeric_columns.groupby(original_data['Cluster']).mean()

# Display the mean true values by cluster
print(true_cluster_summary)


In [ ]:
cluster_df['Prediction'] = xgb_model.predict(X_scaled)
cluster_accuracy = cluster_df.groupby('Cluster').apply(lambda x: (x['Prediction'] == x['RuptureStatus']).mean())
print(cluster_accuracy)


In [ ]:
import shap
import seaborn as sns
shap.initjs()

# Assuming you used StandardScaler for scaling the features
original_values = scaler.inverse_transform(X_scaled[:1000])

# Generate the force plot using original feature values
shap.force_plot(
    explainer.expected_value,
    shap_values.values[:1000],
    original_values,
    feature_names=X.columns
)

# Save the force plot with original feature values
shap.save_html('force_plot.html', shap.force_plot(
    explainer.expected_value,
    shap_values.values[:1000],
    original_values,
    feature_names=X.columns
))


In [ ]:
X.columns

In [ ]:
# Add 'aneurysmLocation' and 'aneurysmType' from the original data to X
X['aneurysmLocation'] = data['aneurysmLocation']
X['aneurysmType'] = data['aneurysmType']

# Drop 'aneurysmLocation_BAS' and 'aneurysmLocation_ICA' to maintain the same number of features
X = X.drop(columns=['aneurysmLocation_BAS', 'aneurysmLocation_ICA'])

# Verify the columns have been updated
print(X.columns)


In [ ]:
shap.dependence_plot('vesselDiameter', shap_values.values, X, interaction_index='aneurysmLocation', feature_names=X.columns)


In [ ]:
shap.dependence_plot('vesselDiameter', shap_values.values, X, interaction_index='aneurysmType', feature_names=X.columns)

"""
(aneurysmType
 LAT    3.251116
 TER    1.892105
 Name: vesselDiameter, dtype: float64,
 'TER',
 'LAT')
 The red points in the SHAP dependence plot correspond to the aneurysm type with the lower mean vesselDiameter, which is TER.
The blue points correspond to the aneurysm type with the higher mean vesselDiameter, which is LAT.
"""

In [ ]:
shap.dependence_plot('neckVesselAngle', shap_values.values, X, interaction_index='aneurysmType', feature_names=X.columns)


In [ ]:
X.columns

In [ ]:
shap.dependence_plot('tortuosity', shap_values.values, X, interaction_index='neckVesselAngle', feature_names=X.columns)
shap.dependence_plot('meanTorsion', shap_values.values, X, interaction_index='tortuosity', feature_names=X.columns)


In [ ]:
# Display all features in the SHAP summary plot
shap.summary_plot(shap_values.values, features=X, feature_names=X.columns, plot_type="layered_violin")


In [ ]:
from skrules import SkopeRules


# Add cluster labels to the original dataset
original_data = data[characteristics].copy()
original_data['Cluster'] = cluster_labels

# Select only numeric columns for rule generation
numeric_columns = original_data.select_dtypes(include=np.number).columns
numeric_data = original_data[numeric_columns]

# Include the cluster labels in the rule generation dataset
numeric_data['Cluster'] = cluster_labels
from sklearn.impute import SimpleImputer

# Impute missing values for numeric data
imputer = SimpleImputer(strategy='mean')  # Use 'mean' to fill missing values
numeric_data_imputed = pd.DataFrame(imputer.fit_transform(numeric_data), 
                                    columns=numeric_data.columns)

# Ensure the Cluster column is properly aligned
numeric_data_imputed['Cluster'] = cluster_labels


In [ ]:
from skrules import SkopeRules

rules_per_cluster = {}

# Loop through each cluster to generate rules
for cluster in np.unique(cluster_labels):
    print(f"Generating rules for Cluster {cluster}...")
    
    # Create a binary target for the current cluster
    numeric_data_imputed['IsCluster'] = (numeric_data_imputed['Cluster'] == cluster).astype(int)
    
    # Initialize and train SkopeRules
    skope = SkopeRules(max_depth=4, n_estimators=30, precision_min=0.5)
    skope.fit(numeric_data_imputed.drop(columns=['Cluster', 'IsCluster']), numeric_data_imputed['IsCluster'])
    
    # Extract and store the rules
    rules = skope.rules_
    rules_per_cluster[cluster] = rules

# Print the rules for each cluster
for cluster, rules in rules_per_cluster.items():
    print(f"\nCluster {cluster} Rules:")
    for rule in rules:
        print(rule)


In [ ]:
import re

# Manually define the feature mapping
feature_map = {
    "__C__0": "age",
    "__C__1": "sacVolume",
    "__C__2": "sacSurfaceArea",
    "__C__3": "vdcVolume",
    "__C__4": "vdcSurfaceArea",
    "__C__5": "sacSectionArea",
    "__C__6": "ellipsoidVolume",
    "__C__7": "ellipsoidMaxSemiaxis",
    "__C__8": "ellipsoidMidSemiaxis",
    "__C__9": "ellipsoidMinSemiaxis",
    "__C__10": "sacCenterlineLength",
    "__C__11": "ostiumSectionArea",
    "__C__12": "ostiumSectionPerimeter",
    "__C__13": "ostiumMinSize",
    "__C__14": "ostiumMaxSize",
    "__C__15": "ostiumShapeFactor",
    "__C__16": "aspectRatio_star",
    "__C__17": "sizeRatio_star",
    "__C__18": "vesselDiameter",
    "__C__19": "neckVesselAngle",
    "__C__20": "sacVesselAngle",
    "__C__21": "meanRadius",
    "__C__22": "meanCurvature",
    "__C__23": "meanTorsion",
    "__C__24": "tortuosity",
    "__C__25": "minRadius",
    "__C__26": "maxRadius",
    "__C__27": "maxCurvature",
    "__C__28": "maxTorsion",
    "__C__29": "bifurcationAngleInPlane",
    "__C__30": "bifurcationAngleOutOfPlane"
}

# Function to replace placeholders with feature names using exact matches
def replace_placeholders(rule, feature_map):
    condition, stats = rule
    for placeholder, feature_name in feature_map.items():
        # Use regex to replace exact matches of placeholders only
        condition = re.sub(rf'\b{re.escape(placeholder)}\b', feature_name, condition)
    return f"{condition} (Support: {stats[2]}, Precision: {stats[0]}, Recall: {stats[1]})"

# Replace placeholders for all clusters
for cluster, rules in rules_per_cluster.items():
    print(f"\nCluster {cluster} Mapped Rules:")
    for rule in rules:
        print(replace_placeholders(rule, feature_map))


In [ ]:
print(numeric_data_imputed.columns)


In [ ]:
print(feature_map)


In [ ]:
for cluster, rules in rules_per_cluster.items():
    print(f"Raw Rules for Cluster {cluster}: {rules}")


In [ ]:
# Rank rules by precision and support
ranked_rules = {}

for cluster, rules in rules_per_cluster.items():
    ranked_rules[cluster] = sorted(
        rules, key=lambda x: (x[1][0], x[1][2]), reverse=True  # Sort by precision, then support
    )

# Display top-ranked rules for each cluster
for cluster, rules in ranked_rules.items():
    print(f"\nTop Rules for Cluster {cluster}:")
    for rule in rules[:5]:  # Display top 5 rules
        print(replace_placeholders(rule, feature_map))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot feature distributions 
features_to_plot = ['vesselDiameter', 'neckVesselAngle', 'ellipsoidVolume', 'sacSectionArea', 'bifurcationAngleInPlane', 'bifurcationAngleOutOfPlane']
for feature in features_to_plot:
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=original_data, x='Cluster', y=feature)
    plt.title(f'{feature} Distribution by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel(feature)
    plt.show()
